# 실행 순서 및 체크 리스트
1. bs4,selenium install
2. 필요 파일 : (1)성엽님이 올려주신 merged_cases.html (2)chromedrive.exe
3. choice1로 '판례_목록.csv' 추출
4. choice2로 크롤링 시작 (시작 판례 번호 기입 가능)

In [1]:
! pip install bs4 selenium

  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
   ---------------------------------------- 0.0/9.6 MB ? eta -:--:--
   --- ------------------------------------ 0.8/9.6 MB 5.6 MB/s eta 0:00:02
   ------- -------------------------------- 1.8/9.6 MB 5.0 MB/s eta 0:00:02
   --------------- ------------------------ 3.7/9.6 MB 6.1 MB/s eta 0:00:01
   -------------------- ------------------- 5.0/9.6 MB 6.4 MB/s eta 0:00:01
   ----------------------------- ---------- 7.1/9.6 MB 7.2 MB/s eta 0:00:01
   -------------------------------------- - 9.2/9.6 MB 7.7 MB/s eta 0:00:01
   ---------------------------------------- 9.6/9.6 MB 7.8 MB/s eta 0:00:00
Using cached urllib3-2.5.0-py3-none-any.whl (129 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.3.0
    Uninstalling urllib3-2.3.0:
      Successfully uninstalled urllib3-2.3.0
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.12.2
    Uninstalling typing_ex

## CSV 파일 생성

In [2]:
from bs4 import BeautifulSoup
import urllib.parse as urlparse
import csv

html_file = "./merged_cases.html"
out_csv = "./판례_목록.csv"

with open(html_file, "r", encoding="utf-8") as f:
    soup = BeautifulSoup(f.read(), "html.parser")

rows = []
for iframe in soup.find_all("iframe"):
    src = iframe.get("src")
    if not src:
        continue
    q = urlparse.urlparse(src).query
    prec = urlparse.parse_qs(q).get("precSeq", [""])[0]
    if prec:
        rows.append({"case_id": prec, "url": src})

with open(out_csv, "w", encoding="utf-8", newline="") as f:
    w = csv.DictWriter(f, fieldnames=["case_id", "url"])
    w.writeheader()
    w.writerows(rows)

print(f"CSV 생성 완료: {out_csv}, {len(rows)}건")

CSV 생성 완료: ./판례_목록.csv, 1268건


## 크롤링 코드

In [4]:
# ========== 통합 크롤링 시스템 ==========
# 판례 데이터 지능형 크롤링 (CSV 시작 인덱스 + 멀티프로세싱 + 속도/ETA + 실패 목록)
# BeautifulSoup + Selenium 하이브리드 방식 (ProcessPoolExecutor)

from bs4 import BeautifulSoup
import requests
import time
import os
import csv
import urllib.parse as urlparse
from datetime import datetime
import re
import math
from concurrent.futures import ThreadPoolExecutor, as_completed
import multiprocessing

# Selenium 관련
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# ===== 기본 경로 설정 =====
LIST_CSV = "./판례_목록.csv"          # CSV 경로는 이 변수로만 관리
OUTPUT_FOLDER = "./판례_크롤링_결과"

# ========== 설정 ==========
HEADERS = {"User-Agent": "Mozilla/5.0"}
DEFAULT_TARGET_CLASSES = ["bo_head type2", "bo_body", "board_right_wrap"]
MIN_LEN = 100  # 최소 텍스트 길이

# ========== 유틸 ==========
def _ensure_dir(path: str):
    if path and not os.path.exists(path):
        os.makedirs(path)

def _sanitize_text(s: str) -> str:
    s = re.sub(r'20%\s*40%\s*60%\s*80%\s*100%', '', s)
    s = re.sub(r'\n\s*\n', '\n', s)
    return s.strip()

def _humanize_seconds(sec: float) -> str:
    if not math.isfinite(sec) or sec < 0:
        return "계산중"
    sec = int(sec)
    m, s = divmod(sec, 60)
    h, m = divmod(m, 60)
    if h:  return f"{h}h {m}m {s}s"
    if m:  return f"{m}m {s}s"
    return f"{s}s"

# ========== BeautifulSoup 시도 ==========
def extract_BeautifulSoup(case_id: str, url: str, output_folder: str):
    """
    BeautifulSoup을 사용한 기본 크롤링 (리다이렉트 없는 경우)
    Returns: (success: bool, message: str, char_len: int)
    """
    try:
        resp = requests.get(url, headers=HEADERS, allow_redirects=False, timeout=30)
        # 리다이렉트면 실패(셀레니움으로 넘김)
        if resp.status_code in [301, 302, 303, 307, 308]:
            return False, "리다이렉트 감지됨", 0

        if resp.status_code != 200:
            return False, f"HTTP 오류: {resp.status_code}", 0

        soup = BeautifulSoup(resp.text, "html.parser")
        v = soup.find(class_="viewwrap")
        if v:
            text = v.get_text(separator="\n")
        else:
            text = soup.get_text(separator="\n")
        text = _sanitize_text(text)

        if len(text) < MIN_LEN:
            return False, "텍스트 너무 짧음", 0

        folder = os.path.join(output_folder, "BeautifulSoup_결과")
        _ensure_dir(folder)
        filename = os.path.join(folder, f"판례_{case_id}.txt")
        with open(filename, "w", encoding="utf-8") as f:
            f.write("=" * 80 + "\n")
            f.write(f"추출 방법: BeautifulSoup (기본 크롤링)\n")
            f.write(f"판례번호: {case_id}\n")
            f.write(f"원본 URL: {url}\n")
            f.write(f"추출 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write(f"텍스트 길이: {len(text):,} 문자\n")
            f.write(f"리다이렉트 여부: 없음\n")
            f.write("=" * 80 + "\n\n")
            f.write(text)

        return True, f"성공: {len(text):,} 문자 추출", len(text)

    except Exception as e:
        return False, f"오류: {e}", 0

# ========== Selenium 시도 ==========
def _new_driver():
    """
    각 프로세스에서 독립적으로 Chrome 드라이버 생성.
    시스템 드라이버 우선, 실패 시 webdriver_manager 시도.
    """
    options = Options()
    options.add_argument("--headless=new")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1920,1080")

    # 방법 1: 시스템에 설치된 chromedriver 사용
    try:
        return webdriver.Chrome(options=options)
    except Exception:
        pass

    # 방법 2: webdriver_manager 사용
    try:
        from webdriver_manager.chrome import ChromeDriverManager
        service = Service(ChromeDriverManager().install())
        return webdriver.Chrome(service=service, options=options)
    except Exception as e:
        raise RuntimeError(f"Chrome 드라이버 초기화 실패: {e}")

def extract_css_classes_selenium(case_number: str, target_classes=None):
    """
    Selenium으로 특정 판례에서 지정된 클래스 텍스트 추출
    Returns: (sections: dict[str,str], total_len: int)
    """
    if target_classes is None:
        target_classes = DEFAULT_TARGET_CLASSES

    url = f"https://www.law.go.kr/LSW/precInfoP.do?precSeq={case_number}&mode=0"
    driver = None
    try:
        driver = _new_driver()
        driver.get(url)

        # body 등장 대기 + JS 로딩(사이트 특성상 약간의 대기 필요)
        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
        time.sleep(6)

        soup = BeautifulSoup(driver.page_source, "html.parser")

        sections = {}
        total_len = 0
        for cls in target_classes:
            elems = soup.find_all(class_=cls)
            key = cls.replace(" ", "_")
            if not elems:
                sections[key] = ""
                continue
            buf = []
            for e in elems:
                t = e.get_text(separator="\n", strip=True)
                t = _sanitize_text(t)
                if t:
                    buf.append(t)
            joined = "\n\n".join(buf).strip()
            sections[key] = joined
            total_len += len(joined)
        return sections, total_len

    except Exception:
        # 실패 시 빈 결과
        return {}, 0
    finally:
        if driver:
            try:
                driver.quit()
            except:
                pass

# ========== CSV 로드 ==========
def load_entries_from_csv(in_path: str):
    """
    CSV에서 (case_id, url) 목록 불러오기
    """
    entries = []
    with open(in_path, "r", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            case_id = str(row.get("case_id", "")).strip()
            url = str(row.get("url", "")).strip()
            if case_id and url:
                entries.append((case_id, url))
    print(f"CSV에서 {len(entries)}건 로드")
    return entries

# ========== 1건 처리(멀티프로세스용) ==========
def process_one(entry):
    """
    멀티프로세싱 워커 함수.
    entry: (case_id, url)
    반환: (case_id, url, success_bool)
    """
    case_id, url = entry
    try:
        # 1) BeautifulSoup 시도
        ok, msg, chars = extract_BeautifulSoup(case_id, url, OUTPUT_FOLDER)
        if ok:
            return (case_id, url, True)

        # 2) Selenium 시도 (리다이렉트 페이지 등)
        sections, total_len = extract_css_classes_selenium(case_id)
        if total_len >= MIN_LEN and any(sections.values()):
            folder = os.path.join(OUTPUT_FOLDER, "Selenium_결과")
            _ensure_dir(folder)
            filename = os.path.join(folder, f"판례_{case_id}.txt")
            with open(filename, "w", encoding="utf-8") as f:
                f.write("=" * 80 + "\n")
                f.write(f"추출 방법: Selenium + CSS 클래스별 추출\n")
                f.write(f"판례번호: {case_id}\n")
                f.write(f"원본 URL: {url}\n")
                f.write(f"추출 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
                f.write(f"총 텍스트 길이: {total_len:,} 문자\n")
                f.write(f"리다이렉트 여부: 있음 (Selenium 사용)\n")
                f.write(f"추출된 CSS 클래스: {', '.join(sections.keys())}\n")
                f.write("=" * 80 + "\n\n")
                for class_name, content in sections.items():
                    f.write(f"[{class_name}]\n")
                    f.write("-" * 40 + "\n")
                    f.write(content if content else "내용 없음")
                    f.write("\n" + "=" * 80 + "\n\n")
            return (case_id, url, True)

        # 둘 다 실패
        return (case_id, url, False)

    except Exception:
        return (case_id, url, False)

# ========== 멀티프로세싱 실행 (진행률/속도/ETA + 실패 목록) ==========
def crawl_from_csv_mp(list_path: str, start_index: int = 0, max_workers: int = 4):
    entries = load_entries_from_csv(list_path)
    if not entries:
        print("CSV에서 불러온 항목이 없습니다.")
        return

    if start_index < 0 or start_index >= len(entries):
        print(f"❌ 인덱스 범위를 벗어났습니다. 0부터 시작합니다.")
        start_index = 0

    slice_entries = entries[start_index:]
    total = len(slice_entries)
    print(f"총 {len(entries)}건 중 {start_index}번 인덱스부터 시작합니다. (처리 대상: {total}건)")
    _ensure_dir(OUTPUT_FOLDER)

    success_cnt = 0
    fail_cnt = 0
    done = 0  # 진행 개수
    failed_ids = []  # 실패한 판례번호 모으기
    start_ts = time.time()

    with ThreadPoolExecutor(max_workers=max_workers) as exe:
        fut_to_idx = {exe.submit(process_one, entry): i for i, entry in enumerate(slice_entries)}
        for fut in as_completed(fut_to_idx):
            done += 1
            try:
                case_id, url, ok = fut.result()
            except Exception:
                case_id, url, ok = ("<unknown>", "<unknown>", False)

            # 속도/ETA 계산
            elapsed = time.time() - start_ts
            rate = (done / elapsed) if elapsed > 0 else 0.0
            remaining = total - done
            eta_sec = (remaining / rate) if rate > 0 else float("inf")

            # 진행률 + 최소 로그 + 속도/ETA
            # 예시 형식: [40/1268] <url>    성공 | 0.85건/초 | 남은시간 ~ 23m 10s
            print(f"[{done}/{total}] {url}    {'성공' if ok else '실패'} | {rate:.2f}건/초 | 남은시간 ~ {_humanize_seconds(eta_sec)}")

            if ok:
                success_cnt += 1
            else:
                fail_cnt += 1
                failed_ids.append(str(case_id))

    print("\n" + "=" * 60)
    print(f"완료: 총 {total}건 처리")
    print(f"성공: {success_cnt}건 / 실패: {fail_cnt}건")
    if total:
        print(f"성공률: {success_cnt/total*100:.1f}%")
    print(f"저장 폴더: {os.path.abspath(OUTPUT_FOLDER)}")

    # 실패 목록 한꺼번에 출력
    if failed_ids:
        print("\n실패한 판례번호 목록:")
        print(", ".join(failed_ids))
    else:
        print("\n실패한 판례 없음 🎉")

# ========== (선택) HTML 파싱 원형 유지하고 싶으면 사용 ==========
def parse_html_file(input_file="./data/merged_cases.html"):
    with open(input_file, "r", encoding="utf-8") as file:
        soup = BeautifulSoup(file, "html.parser")
    iframe_list = soup.find_all("iframe")
    entries = []
    for iframe in iframe_list:
        src = iframe.get("src")
        if src and "precSeq" in src:
            query = urlparse.urlparse(src).query
            params = urlparse.parse_qs(query)
            case_id = params.get("precSeq", [""])[0]
            entries.append((case_id, src))
    print(f"📋 총 {len(entries)}개의 판례 링크를 추출했습니다.")
    return entries

# ========== 메인 ==========
def main():
    print("🚀 판례 통합 크롤링 (CSV 시작 인덱스 + 멀티프로세싱)")
    print("=" * 60)
    print("1) CSV 목록 → 멀티프로세싱 크롤링 실행")
    print("2) (옵션) HTML 파싱만 실행 (원형)")

    choice = input("선택 (1/2): ").strip()

    if choice == "1":
        list_csv = LIST_CSV
        try:
            start_index = int(input("시작 인덱스 입력 (기본 0): ").strip() or "0")
        except ValueError:
            start_index = 0
        try:
            max_workers = int(input(f"동시 프로세스 수 입력 (기본 4): ").strip() or "4")
        except ValueError:
            max_workers = 4

        # 안전장치: 프로세스 과다 방지
        cpu_cnt = max(1, multiprocessing.cpu_count() - 1)
        if max_workers > cpu_cnt * 2:
            print(f"⚠️ 프로세스 수가 많습니다. {cpu_cnt*2} 이하로 자동 조정합니다.")
            max_workers = cpu_cnt * 2

        crawl_from_csv_mp(list_csv, start_index=start_index, max_workers=max_workers)

    elif choice == "2":
        _ = parse_html_file()
    else:
        print("잘못된 선택입니다.")

if __name__ == "__main__":
    main()


🚀 판례 통합 크롤링 (CSV 시작 인덱스 + 멀티프로세싱)
1) CSV 목록 → 멀티프로세싱 크롤링 실행
2) (옵션) HTML 파싱만 실행 (원형)
CSV에서 1268건 로드
총 1268건 중 0번 인덱스부터 시작합니다. (처리 대상: 1268건)
[1/1268] https://www.law.go.kr/LSW/precInfoP.do?precSeq=147646&mode=0    성공 | 2.78건/초 | 남은시간 ~ 7m 36s
[2/1268] https://www.law.go.kr/LSW/precInfoP.do?precSeq=147388&mode=0    성공 | 4.49건/초 | 남은시간 ~ 4m 42s
[3/1268] https://www.law.go.kr/LSW/precInfoP.do?precSeq=136542&mode=0    성공 | 6.73건/초 | 남은시간 ~ 3m 7s
[4/1268] https://www.law.go.kr/LSW/precInfoP.do?precSeq=141833&mode=0    성공 | 8.97건/초 | 남은시간 ~ 2m 20s
[5/1268] https://www.law.go.kr/LSW/precInfoP.do?precSeq=157591&mode=0    성공 | 8.69건/초 | 남은시간 ~ 2m 25s
[6/1268] https://www.law.go.kr/LSW/precInfoP.do?precSeq=148245&mode=0    성공 | 8.49건/초 | 남은시간 ~ 2m 28s
[7/1268] https://www.law.go.kr/LSW/precInfoP.do?precSeq=153152&mode=0    성공 | 9.37건/초 | 남은시간 ~ 2m 14s
[8/1268] https://www.law.go.kr/LSW/precInfoP.do?precSeq=158078&mode=0    성공 | 10.71건/초 | 남은시간 ~ 1m 57s
[9/1268] https://www.law.go.kr/LSW/precInfo

### 실패한 크롤링 파일들에 대해 크롤링 재시도

In [5]:
import csv
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

# 실패한 판례번호만 재시도
failed_ids = [
    "296948", "296976", "297148", "297484",
    "301482", "301640", "301784", "302482",
    "302542", "305242", "305546", "329734",
]

# CSV에서 URL 매핑
id_to_url = {}
with open(LIST_CSV, "r", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        cid = str(row.get("case_id", "")).strip()
        url = str(row.get("url", "")).strip()
        if cid and url:
            id_to_url[cid] = url

entries = [(cid, id_to_url[cid]) for cid in failed_ids if cid in id_to_url]
missing = [cid for cid in failed_ids if cid not in id_to_url]

print(f"재시도 대상: {len(entries)}건")
if missing:
    print(f"CSV에 없음(건너뜀): {', '.join(missing)}")

# 실행
max_workers = 4
success_cnt = 0
fail_ids = []
start_ts = time.time()

def _progress(done: int, total: int, rate: float):
    remaining = total - done
    eta_sec = (remaining / rate) if rate > 0 else float("inf")
    return _humanize_seconds(eta_sec)

with ThreadPoolExecutor(max_workers=max_workers) as exe:
    fut_to_entry = {exe.submit(process_one, e): e for e in entries}
    total = len(fut_to_entry)
    done = 0
    for fut in as_completed(fut_to_entry):
        done += 1
        case_id, url = fut_to_entry[fut]
        try:
            cid, u, ok = fut.result()
        except Exception:
            ok = False
        elapsed = time.time() - start_ts
        rate = (done / elapsed) if elapsed > 0 else 0.0
        print(f"[{done}/{total}] {url}    {'성공' if ok else '실패'} | {rate:.2f}건/초 | 남은시간 ~ {_progress(done, total, rate)}")
        if ok:
            success_cnt += 1
        else:
            fail_ids.append(case_id)

print("\n" + "=" * 60)
print(f"재시도 완료: 총 {len(entries)}건")
print(f"성공: {success_cnt}건 / 실패: {len(fail_ids)}건")
if fail_ids:
    print("실패한 판례번호:")
    print(", ".join(fail_ids))
else:
    print("실패한 판례 없음 🎉")


재시도 대상: 12건
[1/12] https://www.law.go.kr/LSW/precInfoP.do?precSeq=297148&mode=0    성공 | 0.09건/초 | 남은시간 ~ 2m 3s
[2/12] https://www.law.go.kr/LSW/precInfoP.do?precSeq=296976&mode=0    성공 | 0.18건/초 | 남은시간 ~ 56s
[3/12] https://www.law.go.kr/LSW/precInfoP.do?precSeq=296948&mode=0    성공 | 0.26건/초 | 남은시간 ~ 34s
[4/12] https://www.law.go.kr/LSW/precInfoP.do?precSeq=297484&mode=0    성공 | 0.34건/초 | 남은시간 ~ 23s
[5/12] https://www.law.go.kr/LSW/precInfoP.do?precSeq=301482&mode=0    성공 | 0.22건/초 | 남은시간 ~ 31s
[6/12] https://www.law.go.kr/LSW/precInfoP.do?precSeq=301640&mode=0    성공 | 0.27건/초 | 남은시간 ~ 22s
[7/12] https://www.law.go.kr/LSW/precInfoP.do?precSeq=301784&mode=0    성공 | 0.31건/초 | 남은시간 ~ 16s
[8/12] https://www.law.go.kr/LSW/precInfoP.do?precSeq=302482&mode=0    성공 | 0.35건/초 | 남은시간 ~ 11s
[9/12] https://www.law.go.kr/LSW/precInfoP.do?precSeq=302542&mode=0    성공 | 0.27건/초 | 남은시간 ~ 10s
[10/12] https://www.law.go.kr/LSW/precInfoP.do?precSeq=305242&mode=0    성공 | 0.30건/초 | 남은시간 ~ 6s
[11/12] https://

## 판례 위치 검색

In [ ]:
import csv

LIST_CSV = "./판례_목록.csv"

def nth_of_case(case_id, list_csv=LIST_CSV):
    """
    해당 case_id가 CSV에서 몇 번째(1-based)인지 반환.
    없으면 None 반환. URL도 함께 돌려줌.
    """
    target = str(case_id).strip()
    with open(list_csv, "r", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for idx1, row in enumerate(reader, start=1):
            if str(row.get("case_id", "")).strip() == target:
                return idx1, row.get("url", "")
    return None, None

def all_positions_of_case(case_id, list_csv=LIST_CSV):
    """
    중복 대비: 해당 case_id의 모든 위치를 반환.
    반환값: [(index0, index1, url), ...]
      - index0: 0-based 인덱스
      - index1: 1-based 순번(= 몇 번째)
    """
    target = str(case_id).strip()
    results = []
    with open(list_csv, "r", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for i0, row in enumerate(reader):  # 0-based
            if str(row.get("case_id", "")).strip() == target:
                results.append((i0, i0 + 1, row.get("url", "")))
    return results

n, url = nth_of_case("241679")
print(n, url)  # 예: 523 https://...

positions = all_positions_of_case("241679")
for i0, i1, u in positions:
    print(f"0-based={i0}, 1-based={i1}, url={u}")


132 https://www.law.go.kr/LSW/precInfoP.do?precSeq=241679&mode=0
0-based=131, 1-based=132, url=https://www.law.go.kr/LSW/precInfoP.do?precSeq=241679&mode=0
